### 2 Bookshelf Manipulation

In [213]:
import robotic as ry
import random
import time
speed = 0.1
selected_color = [1, 0, 1]
orig_color = [0.80, 0.80, 0.80]

### 1. Add book_shelf.g and 

In [214]:
C = ry.Config()
print(ry.raiPath(''))
C.addFile(ry.raiPath('book_shelf.g'))

C.view()
qHome = C.getJointState()
gripper_orig_pos = C.getFrame("l_gripper").getPosition()
gripper_orig_quat = C.getFrame("l_gripper").getQuaternion()
# print(C.getFrameNames())

objects = ["book1", "book2", "book3", "book4", "mug"]

/home/vboxuser/Desktop/CS449/rai_venv/lib/python3.8/site-packages/robotic/rai-robotModels/


### 2. Randomly select an object + change color of selected obj (to pink)

In [215]:
# select an object randomly
# curObj = random.choice(objects)

curObj = "mug"

print("Selected object: ", curObj)

# color the selected object
C.getFrame(curObj).setColor(selected_color)

# determine orientation of object
obj_z_height = C.getFrame(curObj).getSize()[2]
obj_y_width = C.getFrame(curObj).getSize()[1]
obj_x_length = C.getFrame(curObj).getSize()[0]

if min(obj_z_height, obj_y_width, obj_x_length) == obj_z_height:
  obj_orientation = "horizontal"
elif min(obj_z_height, obj_y_width, obj_x_length) == obj_y_width:
  obj_orientation = "vertical"
elif min(obj_z_height, obj_y_width, obj_x_length) == obj_x_length:
  obj_orientation = "sideways"
  
print("Object orientation: ", obj_orientation)


komo = ry.KOMO()
del komo
C.setJointState(qHome)
C.view()

Selected object:  mug
Object orientation:  vertical


0

### 3. Robot grasps object in a natural manner - uses the orientation of the object (calulcated in prev part) to determine how to grasp it

In [216]:
# komo = ry.KOMO(C, phases=1, slicesPerPhase=30, kOrder=1, enableCollisions=False)

komo_1 = ry.KOMO()
komo_1.setConfig(C, True)
komo_1.setTiming(1., 10, 5., 0)

# add collision constraints
komo_1.addObjective([], ry.FS.accumulatedCollisions, [], ry.OT.eq, [1e0])
komo_1.addObjective([], ry.FS.jointLimits, [], ry.OT.ineq)
komo_1.addObjective([], ry.FS.jointState, [], ry.OT.sos, [1e0], qHome)

# calculate gripper and cargo offset (-40% in x direction)
offset_x = -(C.getFrame(curObj) .getSize()[0] * 0.45)
# this objective moves the gripper close to the cargo
komo_1.addObjective(
    times=[],
    feature=ry.FS.positionRel,
    frames=['l_gripper', curObj],
    type=ry.OT.eq,
    scale=[1e1],
    target=[offset_x, 0, 0]
)
# this objective makes the gripper claws face the shelf objects 
komo_1.addObjective(
    times=[0, 1], 
    feature=ry.FS.scalarProductXZ, 
    frames=[curObj, 'l_gripper'], 
    type=ry.OT.eq, 
    scale=[1e1],
    target=[-1]
)


# these 3 objectives align the gripper claws aligned properly with the book
komo_1.addObjective(
    times=[0, 1], 
    feature=ry.FS.scalarProductXX, 
    frames=['l_gripper', curObj], 
    type=ry.OT.eq, 
    scale=[1e1],
)

komo_1.addObjective(
    times=[0, 1],
    feature=ry.FS.scalarProductZZ, 
    frames=['l_gripper', curObj], 
    type=ry.OT.eq, 
    scale=[1e1]
)

komo_1.addObjective(
    times=[0, 1],
    feature=ry.FS.scalarProductYY, 
    frames=['l_gripper', curObj], 
    type=ry.OT.eq, 
    scale=[1e1]
)

# this objective makes the gripper claws vertical

if obj_orientation == "vertical":
    komo_1.addObjective(
        times=[0, 1],
        feature=ry.FS.scalarProductXY, 
        frames=['l_gripper', curObj], 
        type=ry.OT.eq, 
        scale=[1e1],
        target=[1]
    )
elif obj_orientation == "horizontal":
    komo_1.addObjective(
        times=[0, 1],
        feature=ry.FS.scalarProductXY, 
        frames=['l_gripper', curObj], 
        type=ry.OT.eq, 
        scale=[1e1],
        target=[0]
    )
    # this objective re-adjusts the gripper claws' z and curObj's x to be parallel, to prevent angled grip(and thus no collision)
    komo_1.addObjective(
        times=[0, 1],
        feature=ry.FS.scalarProductYY, 
        frames=['l_gripper', curObj], 
        type=ry.OT.eq, 
        scale=[1e2],
        target=[1]
    )

# solve
ret_1 = ry.NLP_Solver(komo_1.nlp(), verbose=0 ) .solve()
print(ret_1)
q_gripper_to_obj = komo_1.getPath()

for t in range(q_gripper_to_obj.shape[0]):
    C.setJointState(q_gripper_to_obj[t])
    C.view(False, f'waypoint {t}')
    time.sleep(speed)

# komo_1.view_play(False, 2)

# if ret_1.feasible:
#     print("komo is feasible")
# else:
#     print("komo is NOT feasible")

{ time: 0.042918, evals: 23, done: 1, feasible: 0, sos: 56.79, f: 0, ineq: 0, eq: 3.12409 }


### 4. Robot places object in goal area

In [217]:
C.attach('l_gripper', curObj)
komo_2 = ry.KOMO()
komo_2.setConfig(C, True)
komo_2.setTiming(1., 10, 5., 0)

# add collision constraints
komo_2.addObjective([], ry.FS.accumulatedCollisions, [], ry.OT.eq, [1e0])
komo_2.addObjective([], ry.FS.jointLimits, [], ry.OT.ineq)
komo_2.addObjective([], ry.FS.jointState, [], ry.OT.sos, [1e0], qHome)

# calculate gripper and goal offset (in z direction)
curObjPos = C.getFrame(curObj).getSize()
if max(curObjPos) == curObjPos[0]:
    goal_offset = [curObjPos[0], 0, 0]
elif max(curObjPos) == curObjPos[1]:
    goal_offset = [0, curObjPos[1], 0]
else:
    goal_offset = [0, 0, curObjPos[2]]


# this objective moves the gripper close to the goal
komo_2.addObjective(
    times=[],
    feature=ry.FS.positionRel,
    frames=['l_gripper', 'goal'],
    type=ry.OT.eq,
    scale=[1e1],
    target=goal_offset
)

komo_2.addObjective(
    times=[],
    feature=ry.FS.positionDiff,
    frames=[curObj, 'goal'],
    type=ry.OT.eq,
    scale=[1e1],
    target=[0, 0, 0.025]
)

komo_2.addObjective(
    times=[],
    feature=ry.FS.scalarProductXZ,
    frames=['l_gripper', "goal"],
    type=ry.OT.eq,
    scale=[1e1],
    target=[0]
)

komo_2.addObjective(
    times=[],
    feature=ry.FS.scalarProductYZ,
    frames=['goal', 'l_gripper'],
    type=ry.OT.eq,
    scale=[1e1],
    target=[0]
)


# solve
ret_2 = ry.NLP_Solver(komo_2.nlp(), verbose=0 ) .solve()
print(ret_2)
q_obj_to_goal = komo_2.getPath()

for t in range(q_obj_to_goal.shape[0]):
    C.setJointState(q_obj_to_goal[t])
    C.view(False, f'waypoint {t}')
    time.sleep(speed)
    
C.getFrame(curObj) .unLink() .setColor(orig_color)

{ time: 0.025129, evals: 18, done: 1, feasible: 0, sos: 17.5871, f: 0, ineq: 0, eq: 7.173 }


### 5. Robot returns to home position

In [218]:
C.setJointState(qHome)
C.view()

0

### Misc functions (ignore)

In [174]:
# reset colors
for obj in objects:
    C.getFrame(obj).setColor(orig_color)
C.view()

0

In [175]:
# C.addFrame("visual") .setPosition(C.getFrame("goal").getPosition()) .setColor([0, 1, 1]) .setShape(ry.ST.marker, [0.7])
# # # C.addFrame("visual_gripper_orig")  .setShape(ry.ST.marker, [0.]).setPosition(gripper_orig_pos) .setQuaternion(gripper_orig_quat) .setColor([0, 1, 0])
# C.addFrame("visual_gripper_final") .setShape(ry.ST.marker, [0.4]) .setPosition(C.getFrame("l_gripper").getPosition()) .setQuaternion(C.getFrame("l_gripper").getQuaternion()) .setColor([0, 1, 0]) 
# C.view()

In [176]:
# # reinitialise komo and arm position
# try:
#     del komo_2
# except:
#     pass
# C.setJointState(q_gripper_to_obj[0])
# C.view()